# Importations

In [6]:
import pandas as pd
import sklearn 
import scipy


# Chargement des données

In [7]:
df = pd.read_csv("../dataset/Viral_Social_Media_Trends.csv")

# Vérification des données manquantes et des doublons

In [8]:
print("nombre de valeurs manquantes")
print(df.isnull().sum())


nombre de valeurs manquantes
Post_ID             0
Platform            0
Hashtag             0
Content_Type        0
Region              0
Views               0
Likes               0
Shares              0
Comments            0
Engagement_Level    0
dtype: int64


In [9]:
print("nombre de doublons : " + str(df.duplicated().sum()))


nombre de doublons : 0


Pas de valeurs manquantes et pas de doublons dans la dataset, il n'y a donc rien à faire ici.

# Nettoyage simple

In [10]:
df.drop(['Post_ID'], axis = 1, inplace=True)

### On mets toutes les variables string au même format : en minuscule et sans espace

In [11]:
for col in ['Platform', 'Content_Type', 'Region']:
    df[col] = df[col].str.lower().str.strip()

In [12]:
df['Hashtag'] = df['Hashtag'].str.lower().str.strip()

### Affichage des données uniques (vérification de l'orthographe)

In [13]:
for col in ['Platform', 'Content_Type', 'Region']:
    print(df[col].value_counts())
    print("----------------------------")

Platform
youtube      1324
tiktok       1260
instagram    1212
twitter      1204
Name: count, dtype: int64
----------------------------
Content_Type
live stream    855
post           853
reel           841
tweet          836
video          828
shorts         787
Name: count, dtype: int64
----------------------------
Region
usa          677
canada       658
uk           647
brazil       641
india        617
australia    602
japan        592
germany      566
Name: count, dtype: int64
----------------------------


# Encodage des variables categorielles 

Ici, on encode les variables catégorielles à l'aide de One Hot encoding 

In [14]:
df = pd.get_dummies(df, columns=["Platform", "Content_Type", "Region"], drop_first=True)

columns_to_convert = [col for col in df.columns if col.startswith(('Platform', 'Content_Type', 'Region'))]

df[columns_to_convert] = df[columns_to_convert].astype(int)

In [15]:
def label_replace(x) :
    if x == "Low" : 
        return 0
    elif x == "Medium" : 
        return 1
    else :
        return 2

df["Engagement_Level"] = df["Engagement_Level"].map(label_replace)

# Encodage des Hashtag

On va appliquer un encodage TF IDF aux Hashtag

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer() 

tfidf_matrix = tfidf.fit_transform(df['Hashtag'])

tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf.get_feature_names_out())

df = df.reset_index(drop=True)  # pour aligner les index
df = pd.concat([df.drop(columns=["Hashtag"]), tfidf_df], axis=1)

# Sauvegarde de la df 

In [17]:
df.to_csv("../dataset/prepared_data_OH.csv", index = False)